In [ ]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("DF2_Practice") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 

spark = configure_spark_with_delta_pip(builder).getOrCreate()

#spark.conf.set("spark.sql.repl.eagerEval.enabled",True) # OK for exploration, not great for performance
#spark.conf.set("spark.sql.repl.eagerEval.truncate", 500)


### Dataframes advanced + Delta Lake

* Dataframes
    * Custom schemas
    * Pivot
    * Window functions
    * Delta Lake

#### Custom schema

In [ ]:
# Let's import a JSON dataset and have a look at the file

import requests
r = requests.get("https://think.cs.vt.edu/corgis/datasets/json/airlines/airlines.json")

print(r.json()[:2])

In [ ]:
# We can also make json into Row objects
from pyspark.sql import Row

rows = (Row(**x) for x in r.json())

for row in rows:
  print(row)
  break

In [ ]:
# creating a dataframe from this JSON string (Python dictionary) gives us many nested MapType columns

airlines_df = spark.createDataFrame(r.json()) # Inferring schema from Python dict was deemed to be deprecated, but is now again accepted starting from Spark 3.1
#airlines_df2 = spark.createDataFrame(Row(**x) for x in r.json()) # another way of creating df from dict: unpacking each JSON element into a spark Row element

display(airlines_df.limit(3))

In [ ]:
airlines_df.printSchema()

In [ ]:
# let's test some functions on map columns

import pyspark.sql.functions as F

display(airlines_df
        .select(F.explode("Airport") # we can try explode - but this does not make sense for this kind of data
                #F.col("Airport").getItem("Code").alias("AirportCode") # we can use getItem to fetch values per key - good for small map columns, not good for large/nested map columns
                #,F.col("Airport").getItem("Name").alias("AirportName")
                #,F.col("Airport")["Code"] # alternative way, referencing Python dictionary style
                #F.col("Airport.Code"),F.col("Airport.Name") # alternative, easier to call but may confuse as it looks like struct. But Airport.* does not work for map
                #F.map_keys("Airport") # array of keys
                #,F.map_values("Airport") # array of values
                ,"*")
        )

In [ ]:
# usually, a struct type of column is more useful and easier to access
# in this case, we need to custom define a schema when reading in the DataFrame.
# this is also recommended for real-life data pipelines, especially in case of large amount of small data files
# potential down-side: missing schema evolution

from pyspark.sql.types import *

# The outer part needs to be a StructType
# A StructType needs to consist of StructFields
# StructFields have 3 parameters: name, type, nullable

# Note: you can remove or add parts of schema
# Note2: name has to match the key/column name in the dataset.

airport_schema = StructType([
  StructField("Airport",StructType([
    StructField("Code", StringType(), True),
    StructField("Name", StringType(), True)
  ]),True), 
  StructField("Statistics",StructType([
    StructField("Carriers", StructType([
      StructField("Names", StringType(), True),
      StructField("Total", IntegerType(), True)
    ]), True),
    StructField("Minutes Delayed", StructType([
      StructField("Late Aircraft", LongType(), True),
      StructField("National Aviation System", LongType(), True),
      StructField("Weather", LongType(), True),
      StructField("Carrier", LongType(), True),
      StructField("Security", LongType(), True),
      StructField("Total", LongType(), True)
    ]), True),
    StructField("Flights", StructType([
      StructField("Delayed", LongType(), True),
      StructField("Diverted", LongType(), True),
      StructField("Cancelled", LongType(), True),
      StructField("On Time", LongType(), True),
      StructField("Total", LongType(), True)
    ]), True),
    StructField("# of Delays", StructType([
      StructField("Late Aircraft", LongType(), True),
      StructField("National Aviation System", LongType(), True),
      StructField("Weather", LongType(), True),
      StructField("Carrier", LongType(), True),
      StructField("Security", LongType(), True)
    ]), True)
  ]),True),
  StructField("Time",StructType([
    StructField("Label", StringType(), True),
    StructField("Month", IntegerType(), True),
    StructField("Year", IntegerType(), True),
    StructField("Month Name", StringType(), True)
  ]),True)
  #StructField("MyNullTimestamp", TimestampType(), True)
])

In [ ]:
# we can now provide this schema as input in our dataframe creation

airport_schema_df = spark.createDataFrame(r.json(), schema=airport_schema) #spark is not inferring schema. This generally runs much faster for big data
airport_schema_df.printSchema()

In [ ]:
display(airport_schema_df)

In [ ]:
# another way to create a schema is using StructType's "add" method

airport_add_schema = (StructType()
                      .add("Airport", StructType()
                          .add("Code", StringType())
                          .add("Name", StringType())
                          )
                      .add("Time", StructType()
                          .add("Month",IntegerType())
                          .add("Year",IntegerType()))
                     )

airport_add_schema_df = spark.createDataFrame(r.json(), schema=airport_add_schema)
display(airport_add_schema_df)

In [ ]:
# third method, raw string
airport_string_schema = "Airport STRUCT<Code: STRING, Name: STRING>, Time STRUCT<Month: INTEGER, Year: INTEGER, Date: INTEGER>" # date will be null, just an example of adding columns which don't exist

airport_string_schema_df = spark.createDataFrame(r.json(), schema=airport_string_schema) 
display(airport_string_schema_df)

In [ ]:
# It is now much easier to navigate and manipulate the fields.

display(airport_schema_df
        .select("Airport.*"
               ,"Statistics.*"
               ,"*"
               )
       )

#### Pivot

In [ ]:
# pivot table - summarizing a more extensive table, i.e. plotting data points as columns
# let's first load in a dataset

airbnb_df = spark.read.parquet("amsterdam-listings-2018-12-06.parquet")
display(airbnb_df)

In [ ]:
# the dataset has many columns. Let's say we are interested in the average prices per city and neighbourhood.
# we are also interested in the size of the place - how many people it accommodates

display(airbnb_df.select("city"
                       , "neighbourhood"
                       , "accommodates"
                       , "price")
       )

In [ ]:
display(airbnb_df
        .select("city", "neighbourhood", "accommodates", "price")
        .groupby("city", "neighbourhood") # "row" 
        .pivot("accommodates") # columns
        .mean("price") # data / values     # possible options: mean, sum, min, max, count
        .na.fill(0) # for filling out null values. 0 for counts/sums
        .orderBy(F.desc("2")) # for ordering
       )

In [ ]:
# another pivot example

df_wiki = spark.read.parquet("pageviews_by_second.parquet")

display(df_wiki)

In [ ]:
display(df_wiki
        .selectExpr("cast(timestamp as date) as date"
                   ,"hour(timestamp) as hour"
                   ,"site"
                   ,"requests")
        .groupBy("date","hour") #"date"
        .pivot("site")
        .sum("requests")
        .orderBy("date","hour") #"date" 
       )

#### SQL window functions

_Note: a "window" can be many different things. Here we talk about classical SQL window functions_

In [ ]:
# let's load in a new dataset and have a look at it
healthcare_df = spark.read.parquet("health_profile_data.snappy.parquet")
display(healthcare_df)

In [ ]:
# we need to import the Window API and instantiate a Window specification object
# we need also pyspark.sql.functions (F) for using aggregations and functions  

from pyspark.sql import Window

window_spec = Window.partitionBy("_id").orderBy("dte") 

display(healthcare_df
       .withColumn("row_num", F.row_number().over(window_spec)) # similarly can use rank and dense_rank
       )

In [ ]:
# you can use multiple window specs in parallel
  
window_spec_by_hr = Window.partitionBy("_id").orderBy("resting_heartrate") #for descending, you can use .orderBy(F.desc("resting_heartrate"))

display(healthcare_df
       .withColumn("row_num", F.row_number().over(window_spec)) # similarly can use rank and dense_rank
       .withColumn("rank_num", F.rank().over(window_spec_by_hr))
       )

In [ ]:
# use lag / lead for viewing "back" or "ahead" within a partition's rows

display(healthcare_df
       .withColumn("lag_resting_heartrate", F.lag("resting_heartrate").over(window_spec)) # use for getting previous/next value in partition. 
       .withColumn("lead_resting_heartrate", F.lead("resting_heartrate", 5, 0).over(window_spec)) # Can define offset and default value
       .withColumn("diffToPrev",F.expr("resting_heartrate - lag_resting_heartrate")) # useful for getting the increase/decrease
       )


In [ ]:
# rolling windows - useful for moving averages, rolling total, etc

window_spec_rolling = Window.partitionBy("_id").orderBy("dte").rowsBetween(Window.unboundedPreceding, Window.currentRow) # rolling aggregations - everything up to current row
window_spec_rolling_last_week = Window.partitionBy("_id").orderBy("dte").rowsBetween(-6, Window.currentRow) # rolling aggregations, use negative integer for previous rows
window_spec_rolling_plusmin2 = Window.partitionBy("_id").orderBy("dte").rowsBetween(-2, 2) # rolling aggregations, use positive integer for next rows

display(healthcare_df
       .withColumn("row_num", F.row_number().over(window_spec)) 
       .withColumn("rolling_avg", F.avg("resting_heartrate").over(window_spec_rolling_last_week)) # using aggregations with window functions
       .withColumn("row_num_sum", F.sum("row_num").over(window_spec_rolling_plusmin2))
       .withColumn("max_BMI_3", F.max("BMI").over(window_spec_rolling_plusmin2))
       )


#### Delta Lake

In [ ]:
# let's load in a small df for demonstration purposes

iso_df = (spark.read
          .option("header","true")
          .option("inferSchema","true")
          .csv("ISOCountryLookup.csv")
         )
display(iso_df)

In [ ]:
# General issue with data lakes / Hive tables / HDFS storage
# Hard to do SQL / Data Warehouse-like updates
# No ACID compliance (Hive has now in newer versions, but not inherently compatible with Spark)

# Delta Lake = open source
# ACID, time-travel, optimized performance, ...

iso_df.write.mode("overwrite").saveAsTable("iso_t") # default outside of Databricks without format
iso_df.write.format("delta").mode("overwrite").saveAsTable("delta_iso_t") 

In [ ]:
display(spark.table("iso_t"))
#display(spark.table("delta_iso_t"))

In [ ]:
# updating using spark sql statements
#spark.sql("UPDATE iso_t SET independentTerritory = 'Yes' WHERE EnglishShortName = 'Antarctica'") # fails, update not supported
spark.sql("UPDATE delta_iso_t SET independentTerritory = 'Yes' WHERE EnglishShortName = 'Antarctica'") # OK

display(spark.table("delta_iso_t"))

In [ ]:
# another way is to use delta API, useful for more advanced usecases and simpler programmability

from delta.tables import *

delta_table = DeltaTable.forName(spark, "delta_iso_t") 

delta_table.update(
  condition = "EnglishShortName = 'Greenland'",
  set = { "independentTerritory": "'Yes'"}
)

In [ ]:
display(spark.sql("DESCRIBE HISTORY delta_iso_t"))

In [ ]:
# view history of table

display(delta_table.history()) 

In [ ]:
## creating dataframe from previous state

#display(spark.sql("DESCRIBE EXTENDED delta_iso_t")) # getting the table path

# timestamp_df = spark.read.format("delta").option("timestampAsOf", "2024-03-22 14:55:00").load("file:/home/jovyan/spark-warehouse/delta_iso_t")
# display(timestamp_df)
#version_df = spark.read.format("delta").option("versionAsOf", 0).load("file:/home/jovyan/spark-warehouse/delta_iso_t")
#display(version_df)

# restoring table to previous state

#delta_table.restoreToTimestamp('2024-03-22 14:55:00') # restore to a specific timestamp
#delta_table.restoreToVersion(0) # restore table to (oldest) version
#display(spark.table("delta_iso_t"))

## Further reading

* Spark SQL Window functions
  * https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/window.html
* Delta Lake: 
  * https://delta.io/
  * https://medium.com/datalex/5-reasons-to-use-delta-lake-format-on-databricks-d9e76cf3e77d

### Tasks

#### Task 1

Employees' dataset: "employees.csv"

Using window functions, find the **employees** who have worked in a specific **department** the *longest* and *shortest* time.

Resulting dataframe should have 3 columns: employee_name, department, employment_duration

Employment_duration should have 2 possible values: 
* **longest**
  * The employee has worked in the department for the longest time. Based on column _active_record_start_
* **shortest**
  * The employee has worked in the department for the shortest time. Based on column _active_record_start_

Resulting dataframe should have total 6 rows.

Example df.take(3):</br>
<table>
  <tr>
    <th>employee_name</th>
    <th>department</th>
    <th>employment_duration</th>
  </tr>
  <tr>
    <td>CISNEROS JR, HERBERT</td>
    <td>OFFICE</td>
    <td>shortest</td>
  </tr>
  <tr>
    <td>CRAVEN, KEVIN J</td>
    <td>OFFICE</td>
    <td>longest</td>
  </tr>
  <tr>
    <td>WRIGHT, RONALD G</td>
    <td>PRODUCTION</td>
    <td>shortest</td>
  </tr>
</table>

Note: </br>
Should be doable with 2 window functions and 2 transformations.

In [ ]:
# your answer